<a href="https://colab.research.google.com/github/varaslaw/AISingersCoverGen/blob/main/Hina_Mod_AICoverGen_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# AICoverGen WebUI (Modded by [Hina](https://discordlookup.com/user/444684887363026974))
Original Repo by [varaslaw](https://github.com/varaslaw/AISingersCoverGen)

Additional Credits to: [Eddy](https://github.com/Eddycrack864) and [GDR](https://github.com/GatienDoesStuff) for finding a fix to colab update

Please do credit if you do any modifications or translations to this colab notebook.

Simply click `Runtime` in the top navigation bar and `Run all`. Wait for the output of the final cell to show the public Gradio url and click on it.

In [ ]:
#@title Clone repository
from IPython.display import clear_output, Javascript
import codecs
import threading
import time
import os # for reasons...
credit = " - This Notebook was originally modified by Hina"
#=======================Auto Edit======================

#@markdown ---
#@markdown Switch between ```-1 0 1``` or ```-12 0 12``` pitch change control

#@markdown This can only be changed once, you need to restart the whole thing if you wanna change it again
Pitch_Change="12" #@param ['1','12']

repo_name = "AISingersCoverGen"
repo_dir = f"/content/{repo_name}"
cloneing = "https://github.com/varaslaw/AISingersCoverGen.git"
Install_To_Drive=False

#====================Use Drive============
if Install_To_Drive==True:
  from google.colab import drive
  drive.mount('/content/drive')

  !git clone $cloneing $repo_name
  !mv $repo_name /content/drive/MyDrive/
  !rm -rf sample_data
  %cd /content/drive/MyDrive/$repo_name
else:
  !git clone $cloneing $repo_name
  !rm -rf sample_data
  %cd /content/
  %cd $repo_name



def update_timer_and_print():
    global timer
    while True:
        hours, remainder = divmod(timer, 3600)
        minutes, seconds = divmod(remainder, 60)
        timer_str = f'{hours:02}:{minutes:02}:{seconds:02}'
        print(f'Timer: {timer_str} ', end='', flush=True)  # Print without a newline
        time.sleep(1)
        timer += 1
timer = 0
threading.Thread(target=update_timer_and_print, daemon=True).start()
#@markdown ---
LongInstall=False #@param {type:"boolean"}
#@markdown When using LongInstall -> This cell may take a while to install, up to 15 minutes if unlucky
#@markdown <br><br>``NOTE! The L̲o̲n̲g̲I̲n̲s̲t̲a̲l̲l̲ toggle is only needed when it fails to use the GPU when making a cover``
if LongInstall==True:
  !sudo apt update
  !yes | sudo DEBIAN_FRONTEND=noninteractive apt-get -yq install cuda-11-8
  clear_output()
  !find / -name '*libcublasLt.so*' #Just to see the versions, you will have 11.8 and 12.2
  time.sleep(10)

clear_output()
print("Done Cloning Repository"+credit)

In [ ]:
#@title Install requirements
import time
#@markdown This cell can take up to 5 minutes to finish
%cd $repo_dir
!python -m pip install -U pip wheel
!python -m pip install -r requirements.txt
!sudo apt update
!sudo apt install -y sox
!python src/download_models.py
print('Finished installing requirements and downloading core models')

In [ ]:
#@title Run WebUI
#@markdown ---
%cd $repo_dir
runpice='src/webui-ru.py'
Url="Gradio" #@param ['Gradio','Ngrok','Cloudflared']
#@markdown ---
#@markdown - Token is only needed if you are using Ngrok. You can make an account on [ngrok](https://dashboard.ngrok.com/signup) for free.
#@markdown - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and place it here:
Token="" #@param {type:"string"}
if Url=="Gradio":
  !python $runpice --share
elif Url == "Cloudflared":
  # Install Cloudflared
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  # Delete log file
  !rm -rf nohup.out
  # Run cloudflared
  !nohup cloudflared tunnel --url localhost:9999 &
  clear_output()

  time.sleep(5)

  !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  !python $runpice --listen-port 9999
else:
  !python -m pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(Token)
  ngrok.kill()
  cover_tunnel = ngrok.connect(9999)
  print("CoverGen URL:", cover_tunnel.public_url)
  print(credit)
  !python $runpice --listen-port 9999

[![](https://i.pinimg.com/474x/de/72/9e/de729ecfa41b69901c42c82fff752414.jpg)](https://discordlookup.com/user/444684887363026974)